In [32]:
import requests
from typing import List, Optional

from pydantic import BaseModel
from poke_env.environment.move import Move
from poke_env.environment.pokemon import Pokemon



class DamageCalcResult(BaseModel):
    damage: List[int]
    # recoilInfo: dict
    # recoveryInfo: dict
    # kochanceInfo: dict


In [33]:
from poke_env.data.gen_data import GenData


TYPE_CHART = GenData(1).load_type_chart(1)

ValueError: GenData for gen 1 already initialized.

In [3]:
class PokemonInfo(BaseModel):
    name: str
    level: int
    ivs: Optional[dict] = None
    evs: Optional[dict] = None

In [61]:
def get_1v1_calc(p1_pkmn: PokemonInfo, p2_pkmn: PokemonInfo, moves: List[str], gen: int =1):
    dmg_moves, no_effect_moves = [], []
    def_pkmn = Pokemon(species=p2_pkmn.name, gen=1)
    for move in moves:
        move_type = Move(move, 1).type.name
        
        def_type_1 = def_pkmn.type_1.name
        if TYPE_CHART[def_type_1][move_type] == 0:
            no_effect_moves.append(move)
            continue
        
        if def_pkmn.type_2:
            def_type_2 = def_pkmn.type_2.name
            if TYPE_CHART[def_type_2][move_type] == 0:
                no_effect_moves.append(move)
                
                continue
        
        dmg_moves.append(move)    

    data = {
        "gen": gen,
        "attackingPokemon": p1_pkmn.name,
        "attackerLevel": p1_pkmn.level,
        "attackerIVs": p1_pkmn.ivs,
        "attackerEVs": p1_pkmn.evs,
        
        "defendingPokemon": p2_pkmn.name,
        "defenderLevel": p2_pkmn.level,
        "defenderIVs": p2_pkmn.ivs,
        "defenderEVs": p2_pkmn.evs,
        
        "moveNames": dmg_moves,
    }
    
    {'damageRange': [0, 0],
    'recoilInfo': {'recoil': [0, 0], 'text': ''},
    'recoveryInfo': {'recovery': [0, 0], 'text': ''}}


    response = requests.post("http://localhost:3000/calculatefull", json=data, timeout=3)
    json = response.json()
    try:
        for obj in json:
            if type(obj["damage"]) == int:
                obj["damage"] = [obj["damage"]]
    except:
        # print(json)
        pass
    # json_class = DamageCalcResult.model_validate(json)
    # return {
    #     move: DamageCalcResult.model_validate(dmg_result)
    #     for move, dmg_result in zip(dmg_moves.values(), json)
    # }
    return json, dmg_moves

In [77]:
abra = PokemonInfo(name="Exeggutor", level=65)
aerodactyl = PokemonInfo(name="Snorlax", level=70)

In [83]:
# get_1v1_calc(abra, aerodactyl, moves=["Psychic", "Seismic Toss", "Thunderbolt", "Ice Punch"])
# get_1v1_calc(abra, aerodactyl, moves=["Mega Drain", "Explosion", "Double Edge"])
get_1v1_calc(abra, aerodactyl, moves=["doubleedge", "psychic", "megadrain"])

({'results': [{'damageRange': [57, 67],
    'recoilInfo': {'recoil': [5.5, 6.4], 'text': '5.5 - 6.4% recoil damage'},
    'recoveryInfo': {'recovery': [0, 0], 'text': ''}},
   {'damageRange': [92, 109],
    'recoilInfo': {'recoil': [0, 0], 'text': ''},
    'recoveryInfo': {'recovery': [0, 0], 'text': ''}},
   {'damageRange': [41, 49],
    'recoilInfo': {'recoil': [0, 0], 'text': ''},
    'recoveryInfo': {'recovery': [21, 25], 'text': '8.1 - 9.6% recovered'}}],
  'attackerStats': {'hp': 258,
   'atk': 188,
   'def': 175,
   'spa': 227,
   'spd': 227,
   'spe': 136},
  'defenderStats': {'hp': 369,
   'atk': 224,
   'def': 161,
   'spa': 161,
   'spd': 161,
   'spe': 112}},
 ['doubleedge', 'psychic', 'megadrain'])

In [92]:
vars(Move("softboiled", 1))

TypeError: vars() argument must have __dict__ attribute

In [64]:
Move("doubleedge", 1).id

'doubleedge'

In [22]:


abra_pkmn = Pokemon(1, details="abra, L81")
abra_pkmn.type_1

<PokemonType.PSYCHIC: 15>

In [75]:
abra_pkmn.species

'abra'

In [12]:
print(abra_pkmn.status)

None


In [13]:
import json
with open("gen1randbats.json") as f:
    data = json.load(f)

In [14]:
abra_data = data["Abra"]
aerodactyl_data = data["Aerodactyl"]
abra_data

{'level': 81,
 'moves': {'Psychic': 1,
  'Seismic Toss': 1,
  'Thunder Wave': 1,
  'Substitute': 0.3381,
  'Reflect': 0.3338,
  'Counter': 0.3281},
 'evs': {'atk': 0},
 'ivs': {'atk': 2}}

In [15]:
aerodactyl_data

{'level': 75,
 'moves': {'Double-Edge': 1,
  'Fire Blast': 1,
  'Hyper Beam': 1,
  'Sky Attack': 0.6656,
  'Agility': 0.3344}}

In [16]:
attacker = "Abra"
defender = "Aerodactyl"

attacker_info = PokemonInfo(name=attacker, level=abra_data["level"], ivs=abra_data["ivs"], evs=abra_data["evs"])
defender_info = PokemonInfo(name=defender, level=aerodactyl_data["level"], ivs=None, evs=None)

moves = list(abra_data["moves"].keys())
clean_move = lambda move: move.replace(" ", "").replace("-", "").lower()
dmg_moves = list(filter(lambda move: Move(move_id=clean_move(move), gen=1).base_power > 0, moves))

dmg_calc = get_1v1_calc(attacker_info, defender_info, dmg_moves)
dmg_calc

{'results': [{'damageRange': [120, 142],
   'recoilInfo': {'recoil': [0, 0], 'text': ''},
   'recoveryInfo': {'recovery': [0, 0], 'text': ''}},
  {'damageRange': [81, 81],
   'recoilInfo': {'recoil': [0, 0], 'text': ''},
   'recoveryInfo': {'recovery': [0, 0], 'text': ''}},
  {'damageRange': [1, 2],
   'recoilInfo': {'recoil': [0, 0], 'text': ''},
   'recoveryInfo': {'recovery': [0, 0], 'text': ''}}],
 'attackerStats': {'hp': 206,
  'atk': 90,
  'def': 104,
  'spa': 250,
  'spd': 250,
  'spe': 226},
 'defenderStats': {'hp': 274,
  'atk': 232,
  'def': 172,
  'spa': 164,
  'spd': 164,
  'spe': 269}}

In [17]:
dmg_moves

['Psychic', 'Seismic Toss', 'Counter']

In [18]:
dmg_calc["results"][0]["damageRange"][0] / dmg_calc["defenderStats"]["hp"]

0.43795620437956206

In [19]:
dmg_calc = get_1v1_calc(attacker_info, defender_info, [])

In [20]:
EXCLUDE_DMG_MOVES = {"counter"}
clean_move = lambda move: move.replace(" ", "").replace("-", "").lower()
dmg_calc_results = {}

counter, inner_counter = 0, 0

for attacker, atk_data in data.items():
    attacker_info = PokemonInfo(name=attacker, 
                                level=atk_data["level"], 
                                ivs=atk_data["ivs"] if "ivs" in atk_data else None, 
                                evs=atk_data["evs"] if "evs" in atk_data else None)
    
    moves = list(atk_data["moves"].keys())
    dmg_moves = list(filter(lambda move: Move(move_id=clean_move(move), gen=1).base_power > 0, moves))
    dmg_moves = list(filter(lambda move: clean_move(move) not in EXCLUDE_DMG_MOVES, dmg_moves))
    
    atk_stats = get_1v1_calc(attacker_info, None, [])
    atk_hp = atk_stats["attackerStats"]["hp"]
    dmg_calc_results[attacker] = {"hp": atk_hp, 
                                  "spe": atk_stats["attackerStats"]["spe"]}
    
    for defender, def_data in data.items():
        defender_info = PokemonInfo(name=defender, 
                                    level=def_data["level"], 
                                    ivs=def_data["ivs"] if "ivs" in def_data else None, 
                                    evs=def_data["evs"] if "evs" in def_data else None)
        
        dmg_calc = get_1v1_calc(attacker_info, defender_info, dmg_moves)
        
        def_hp = dmg_calc["defenderStats"]["hp"]
        dmg_calc_percent = {move: {} for move in dmg_moves}
        for i, entry in enumerate(dmg_calc["results"]):
            dmg_calc_percent[dmg_moves[i]]["damage"] = (entry["damageRange"][0] / def_hp,
                                                        entry["damageRange"][1] / def_hp)
            recoil_lower = entry["recoilInfo"]["recoil"][0]
            recoil_higher = entry["recoilInfo"]["recoil"][1]
            if recoil_lower != 0 or recoil_higher != 0:
                dmg_calc_percent[dmg_moves[i]]["recoil"] = (recoil_lower / def_hp,
                                                            recoil_higher/ def_hp)
            recovery_lower = entry["recoveryInfo"]["recovery"][0]
            recovery_higher = entry["recoveryInfo"]["recovery"][1]
            if recovery_lower != 0 or recovery_higher != 0:
                dmg_calc_percent[dmg_moves[i]]["recovery"] = (recoil_lower/ atk_hp,
                                                            recovery_higher / atk_hp)
            
        for move in dmg_calc_percent:
            for calc in dmg_calc_percent[move]:
                if dmg_calc_percent[move][calc][0] == 0.0 and dmg_calc_percent[move][calc][1] == 0.0:
                    continue
                dmg_calc_percent[move][calc] = (round(dmg_calc_percent[move][calc][0], 4),
                                                round(dmg_calc_percent[move][calc][1], 4))
        
        dmg_calc_results[attacker][defender] = dmg_calc_percent
        
        inner_counter += 1
        if inner_counter == 2:
            break
    
    counter += 1
    if counter == 1:
        break
        

AttributeError: 'NoneType' object has no attribute 'name'

In [ ]:
attacker, defender

('Abra', 'Aerodactyl')

In [ ]:
dmg_calc_results

{'Abra': {'hp': 206,
  'spe': 226,
  'Abra': {'Psychic': {'damage': (0.1893, 0.2282)},
   'Seismic Toss': {'damage': (0.3932, 0.3932)}},
  'Aerodactyl': {'Psychic': {'damage': (0.438, 0.5182)},
   'Seismic Toss': {'damage': (0.2956, 0.2956)}}}}

In [84]:
with open("gen1randbats_dmg_calc.json", "r") as f:
    dmg_calc_results = json.load(f)

In [85]:
import os
file_size = os.path.getsize("gen1randbats_dmg_calc.json")
file_size / 1024 / 1024

12.689742088317871

In [86]:
dmg_calc_results["Exeggutor"]["Snorlax"]

{'explosion': {'damage': [0.5176, 0.6098], 'recoil': [-1, -1]},
 'psychic': {'damage': [0.2493, 0.2954]},
 'doubleedge': {'damage': [0.1545, 0.1816], 'recoil': [-5.5, -6.4]},
 'megadrain': {'damage': [0.1111, 0.1328], 'recovery': [0.0814, 0.0969]},
 'hyperbeam': {'damage': [0.2304, 0.271]}}

In [87]:
import sys
sys.getsizeof(dmg_calc_results) / 1024

3.25